This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [23]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [52]:
# First, import the relevant modules
import requests
import collections
import numpy as np
import timeit

In [48]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
database_code = 'FSE'
dataset_code = 'AFX_X'
url = "https://www.quandl.com/api/v3/datasets/{0}/{1}.json?&start_date=2017-01-01&end_date=2017-12-31&api_key={2}"
res = requests.get(url.format(database_code, dataset_code, API_KEY))
json_res = res.json()

In [49]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_res.keys()
meta_data = json_res['dataset']
data = meta_data.pop('data')


In [50]:
print(len(data))
meta_data

255


{'id': 10095370,
 'dataset_code': 'AFX_X',
 'database_code': 'FSE',
 'name': 'Carl Zeiss Meditec (AFX_X)',
 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
 'refreshed_at': '2020-12-01T14:48:09.907Z',
 'newest_available_date': '2020-12-01',
 'oldest_available_date': '2000-06-07',
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'frequency': 'daily',
 'type': 'Time Series',
 'premium': False,
 'limit': None,
 'transform': None,
 'column_index': None,
 'start_date': '2017-01-01',
 'end_date': '2017-12-31',
 'collapse': None,
 'order': None,
 'database_id': 6129}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [168]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
%time
open_price = [data[i][1]  for i in range(0,len(data)) if  data[i][1] != None ]
print(len(open_price))
max_open_price = max(open_price)
min_open_price = min(open_price)
print( max_open_price, min_open_price)

Wall time: 0 ns
252
53.11 34.0


In [167]:
# using numpy
%time
open_price = np.array(data)[:,1]
open_price[open_price==None] = np.nan
open_price = open_price.astype('float64')

open_price = open_price[~(np.isnan(open_price))]
print(len(open_price))
max_open_price = open_price.max()
min_open_price = open_price.min()
print( max_open_price, min_open_price)

Wall time: 0 ns
252
53.11 34.0


In [151]:
#4. What was the largest change in any one day (based on High and Low price)?
high_low_price = [[data[i][2],data[i][3]]  for i in range(0,len(data)) if (data[i][2] != None) & (data[i][3] !=None)]
print(len(price_change))
price_change = [ h-l for h, l in high_low_price]
max_price_change = max(price_change)
print(max_price_change)

255
2.8100000000000023


In [153]:
#5. What was the largest change between any two days (based on Closing Price)?
closing_price = [data[i][4]  for i in range(0,len(data)) if  data[i][4] != None ]
print(len(closing_price))
closing_price_change = [closing_price[i] -closing_price[i-1] for i in range(1,len(closing_price))]
max_closing_price_change = max(closing_price_change)
print(max_closing_price_change)

255
2.559999999999995


In [155]:
#6. What was the average daily trading volume during this year?
daily_vol = [data[i][6]  for i in range(0,len(data)) if  data[i][6] != None ]
print(len(daily_vol))
ave_daily_vol = sum(daily_vol)/len(daily_vol)
print(ave_daily_vol)

255
89124.33725490196


In [163]:
#7. What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)

if len(daily_vol)%2 == 0:
    median_i = len(daily_vol)/2 -1 
else:
    median_i = len(daily_vol)//2
sorted(daily_vol)[median_i]

76286.0